In [30]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import json
from html import unescape
from datetime import datetime

# 전체 페이지 수 가져오기
def get_page_num():
    url = 'https://www.skinnytaste.com/recipes/vegetarian'
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    max_num = soup.find('div', {'class': 'nav-links'}).contents[-3].text
    nums = list(range(1, int(max_num) + 1))

    return nums

# 입력한 페이지의 전체 레시피 링크 가져오기
def get_links(i):
    link_list = list()
    url = 'https://www.skinnytaste.com/recipes/vegetarian/page/' + str(i)
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    articles = soup.find_all('div', {'class': 'archive-post'})
    for article in articles:
        link_list.append(article.contents[1].get('href'))

    return link_list


# 입력한 링크의 제목, 댓글, 레시피, 재료, 조리시간, 분량, 영양정보, 이미지 가져오기
def get_contents(url):
    print('in - ', url)
    contents = dict()
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    # 제목
    title = soup.find('div', {'class': 'post-title'})
    if title:
        title = title.contents[1].text
    else:
        pass

    title_index = int(title.find('Recipe'))

    if title_index > 0:
        title = title[:title_index]
    else:
        pass

    # 레시피 내용
    recipe_para = soup.find_all('ul', {'class': 'wprm-recipe-instructions'})
    for recipes in recipe_para:
        if recipes:
            recipe = recipes.find_all('li', {'class': 'wprm-recipe-instruction'})
            for r in recipe:
                step = str(int(r.get('id').split('-')[-1]) + 1)
                recipes_list.append(step + '. ' + r.text)
        else:
            pass

    # 재료
    ingredients_list = list()
    ingredients = soup.find_all('li', {'class': 'wprm-recipe-ingredient'})
    for ingredient in ingredients:
        if ingredient:
            ingredients_list.append(ingredient.text)
        else:
            pass

    # 조리시간
    cooktime_list = list()
    cooktimes = soup.find_all('span', {'class': 'wprm-recipe-time wprm-block-text-normal'})
    if cooktimes:
        cooktimes = cooktimes[-1]
        for cooktime in cooktimes:
            if cooktime:
                cooktime_list.append(cooktime.text)
            else:
                pass
    cooktime_list = ''.join(cooktime_list)

    # 분량
    servings = soup.find('span', {'class': 'wprm-recipe-servings-with-unit'})
    if servings:
        servings = servings.text
    else:
        pass

    # 영양정보
    nut_list = list()
    nutrients = soup.find_all('span', {'class': 'wprm-meta-value'})
    for nutrient in nutrients:
        if nutrient:
            nut_list.append(nutrient.text)
        else:
            pass

    calories = ''
    carbs = ''
    protein = ''
    total_fat = ''

    if nut_list:
        calories = nut_list[0].split(' ')[0] + 'kcal'
        carbs = nut_list[2].split(' ')[0] + 'g'
        protein = nut_list[1].split(' ')[0] + 'g'
        total_fat = nut_list[3].split(' ')[0] + 'g'
    else:
        pass


    # 이미지
    image = soup.find('div', {'class', 'wprm-recipe-image wprm-block-image-normal'})
    if image:
        image = image.contents[0].get('data-lazy-srcset').split(' ')[2]
    else:
        pass

    if recipes_list != {}:
        # if comments_num > 4:
        # 주소 (사이트)
        contents['site'] = url
        # 제목
        contents['title'] = title
        # 재료
        contents['ingredients'] = ingredients_list
        # 조리시간
        contents['time'] = cooktime_list
        # 분량
        contents['serving'] = servings
        # 레시피 내용
        contents['recipe'] = recipes_list
        # 칼로리
        contents['calories'] = calories
        # 탄수화물
        contents['carbs'] = carbs
        # 단백질
        contents['protein'] = protein
        # 지방
        contents['total_fat'] = total_fat
        # # 댓글
        # contents['comments'] = com_list
        # 이미지
        contents['image'] = image
        # else:
        #     pass
    else:
        contents = ''

    print('-url: ', url)

    return contents


# 전체 페이지 레시피 댓글 가져오기
def get_all_page_comment(nums):
    total = dict()
    title_comments = list()

    with ThreadPoolExecutor(max_workers=10) as executor:
        for num in nums:
            links = get_links(num)
            for link in links:
                content = executor.submit(get_contents, link)
                title_comments.append(content.result())

    date = datetime.today().strftime("%Y%m%d")
    
    total['date'] = date                
    total['skinnytaste'] = title_comments
    return total


# 메인에서 실행
if __name__ == '__main__':
    nums = get_page_num()
    total = get_all_page_comment(nums)

    date = datetime.today().strftime("%Y%m%d")
        
    with open(f'D:\\fruit_hada\\crawling\\crawling_result\\{date}_skinnytaste_crawling_all.json', 'w', encoding='utf-8-sig') as file:
        json.dump(total, file, indent="\t")            

in -  https://www.skinnytaste.com/whipped-feta-dip/
-url:  https://www.skinnytaste.com/whipped-feta-dip/
in -  https://www.skinnytaste.com/spinach-dip-stuffed-mushrooms/
-url:  https://www.skinnytaste.com/spinach-dip-stuffed-mushrooms/
in -  https://www.skinnytaste.com/slow-cooker-steel-cut-oats/
-url:  https://www.skinnytaste.com/slow-cooker-steel-cut-oats/
in -  https://www.skinnytaste.com/veggie-stromboli/
-url:  https://www.skinnytaste.com/veggie-stromboli/
in -  https://www.skinnytaste.com/butternut-squash-galette/
-url:  https://www.skinnytaste.com/butternut-squash-galette/
in -  https://www.skinnytaste.com/rainbow-quinoa-salad-with-lemon-dressing/
-url:  https://www.skinnytaste.com/rainbow-quinoa-salad-with-lemon-dressing/
in -  https://www.skinnytaste.com/fennel-gratin/
-url:  https://www.skinnytaste.com/fennel-gratin/
in -  https://www.skinnytaste.com/herb-roasted-carrots/
-url:  https://www.skinnytaste.com/herb-roasted-carrots/
in -  https://www.skinnytaste.com/high-protein-z

-url:  https://www.skinnytaste.com/avocado-quinoa-salad/
in -  https://www.skinnytaste.com/light-swiss-chard-frittata/
-url:  https://www.skinnytaste.com/light-swiss-chard-frittata/
in -  https://www.skinnytaste.com/warm-salad-with-artichoke-hearts-roasted-peppers-mozzarella/
-url:  https://www.skinnytaste.com/warm-salad-with-artichoke-hearts-roasted-peppers-mozzarella/
in -  https://www.skinnytaste.com/deviled-eggs/
-url:  https://www.skinnytaste.com/deviled-eggs/
in -  https://www.skinnytaste.com/chocolate-frozen-yogurt-in-the-ninja-creami/
-url:  https://www.skinnytaste.com/chocolate-frozen-yogurt-in-the-ninja-creami/
in -  https://www.skinnytaste.com/lemon-cheesecake-yogurt-cups/
-url:  https://www.skinnytaste.com/lemon-cheesecake-yogurt-cups/
in -  https://www.skinnytaste.com/banana-nut-pancakes/
-url:  https://www.skinnytaste.com/banana-nut-pancakes/
in -  https://www.skinnytaste.com/peanut-butter-breakfast-oatmeal-bowl/
-url:  https://www.skinnytaste.com/peanut-butter-breakfast-

-url:  https://www.skinnytaste.com/protein-pb-j-smoothie-bowl/
in -  https://www.skinnytaste.com/layered-potato-cups-with-spring-herbs-and-leeks/
-url:  https://www.skinnytaste.com/layered-potato-cups-with-spring-herbs-and-leeks/
in -  https://www.skinnytaste.com/spanakopita-baked-eggs/
-url:  https://www.skinnytaste.com/spanakopita-baked-eggs/
in -  https://www.skinnytaste.com/cacio-e-pepe-frittata-with-cauliflower-and-lemony-yogurt/
-url:  https://www.skinnytaste.com/cacio-e-pepe-frittata-with-cauliflower-and-lemony-yogurt/
in -  https://www.skinnytaste.com/hearts-of-palm-noodle-peanut-stir-fry/
-url:  https://www.skinnytaste.com/hearts-of-palm-noodle-peanut-stir-fry/
in -  https://www.skinnytaste.com/flaugnarde-of-mixed-berries-clafoutis/
-url:  https://www.skinnytaste.com/flaugnarde-of-mixed-berries-clafoutis/
in -  https://www.skinnytaste.com/high-protein-oat-waffles/
-url:  https://www.skinnytaste.com/high-protein-oat-waffles/
in -  https://www.skinnytaste.com/cheesy-eggplant-gno

-url:  https://www.skinnytaste.com/orzo-with-zucchini-and-tomato/
in -  https://www.skinnytaste.com/tropical-mango-blueberry-lassi/
-url:  https://www.skinnytaste.com/tropical-mango-blueberry-lassi/
in -  https://www.skinnytaste.com/egg-tomato-and-scallion-sandwich/
-url:  https://www.skinnytaste.com/egg-tomato-and-scallion-sandwich/
in -  https://www.skinnytaste.com/skillet-mixed-berry-buttermilk-cobbler/
-url:  https://www.skinnytaste.com/skillet-mixed-berry-buttermilk-cobbler/
in -  https://www.skinnytaste.com/hot-cross-buns/
-url:  https://www.skinnytaste.com/hot-cross-buns/
in -  https://www.skinnytaste.com/lebanese-lentil-soup/
-url:  https://www.skinnytaste.com/lebanese-lentil-soup/
in -  https://www.skinnytaste.com/baked-rice-and-peas/
-url:  https://www.skinnytaste.com/baked-rice-and-peas/
in -  https://www.skinnytaste.com/banana-bread-muffin-in-a-mug/
-url:  https://www.skinnytaste.com/banana-bread-muffin-in-a-mug/
in -  https://www.skinnytaste.com/kale-and-butternut-squash-s

-url:  https://www.skinnytaste.com/potato-and-green-bean-salad-325-pts/
in -  https://www.skinnytaste.com/green-detox-soup/
-url:  https://www.skinnytaste.com/green-detox-soup/
in -  https://www.skinnytaste.com/air-fryer-hard-boiled-eggs/
-url:  https://www.skinnytaste.com/air-fryer-hard-boiled-eggs/
in -  https://www.skinnytaste.com/avocado-egg-rolls/
-url:  https://www.skinnytaste.com/avocado-egg-rolls/
in -  https://www.skinnytaste.com/cast-iron-thin-crust-pizza/
-url:  https://www.skinnytaste.com/cast-iron-thin-crust-pizza/
in -  https://www.skinnytaste.com/migas/
-url:  https://www.skinnytaste.com/migas/
in -  https://www.skinnytaste.com/skinny-honey-lemon-bars/
-url:  https://www.skinnytaste.com/skinny-honey-lemon-bars/
in -  https://www.skinnytaste.com/buddha-bowl/
-url:  https://www.skinnytaste.com/buddha-bowl/
in -  https://www.skinnytaste.com/string-bean-salad-2-pts/
-url:  https://www.skinnytaste.com/string-bean-salad-2-pts/
in -  https://www.skinnytaste.com/cream-of-asparag

...

-url:  https://www.skinnytaste.com/black-eyed-pea-dip/
in -  https://www.skinnytaste.com/easy-garlic-broccolini/
-url:  https://www.skinnytaste.com/easy-garlic-broccolini/
in -  https://www.skinnytaste.com/dark-chocolate-oatmeal-lace-cookies/
-url:  https://www.skinnytaste.com/dark-chocolate-oatmeal-lace-cookies/
in -  https://www.skinnytaste.com/banana-cranberry-bread/
-url:  https://www.skinnytaste.com/banana-cranberry-bread/
in -  https://www.skinnytaste.com/light-and-easy-cauliflower-gratin/
-url:  https://www.skinnytaste.com/light-and-easy-cauliflower-gratin/
in -  https://www.skinnytaste.com/slow-cooker-garlic-sweet-potato-mash/
-url:  https://www.skinnytaste.com/slow-cooker-garlic-sweet-potato-mash/
in -  https://www.skinnytaste.com/skinny-cranberry-swirl-cheesecake/
-url:  https://www.skinnytaste.com/skinny-cranberry-swirl-cheesecake/
in -  https://www.skinnytaste.com/morning-maple-cranberry-pecan-oat-bars/
-url:  https://www.skinnytaste.com/morning-maple-cranberry-pecan-oat-ba

-url:  https://www.skinnytaste.com/this-savory-quinoa-stuffing-is/
in -  https://www.skinnytaste.com/pumpkin-hazelnut-white-chocolate/
-url:  https://www.skinnytaste.com/pumpkin-hazelnut-white-chocolate/
in -  https://www.skinnytaste.com/potato-parsnip-mash/
-url:  https://www.skinnytaste.com/potato-parsnip-mash/
in -  https://www.skinnytaste.com/pumpkin-cheesecake-shooters/
-url:  https://www.skinnytaste.com/pumpkin-cheesecake-shooters/
in -  https://www.skinnytaste.com/roasted-pumpkin-sage-soup/
-url:  https://www.skinnytaste.com/roasted-pumpkin-sage-soup/
in -  https://www.skinnytaste.com/spooky-spider-cupcakes/
-url:  https://www.skinnytaste.com/spooky-spider-cupcakes/
in -  https://www.skinnytaste.com/garlic-sweet-potato-mash/
-url:  https://www.skinnytaste.com/garlic-sweet-potato-mash/
in -  https://www.skinnytaste.com/garlic-roasted-cauliflower-with-toasted/
-url:  https://www.skinnytaste.com/garlic-roasted-cauliflower-with-toasted/
in -  https://www.skinnytaste.com/delightful-a

-url:  https://www.skinnytaste.com/skinny-no-bake-cookies/
in -  https://www.skinnytaste.com/leftover-parmesan-mashed-potato-patties/
-url:  https://www.skinnytaste.com/leftover-parmesan-mashed-potato-patties/
in -  https://www.skinnytaste.com/make-over-corn-casserole/
-url:  https://www.skinnytaste.com/make-over-corn-casserole/
in -  https://www.skinnytaste.com/roasted-parmesan-green-beans/
-url:  https://www.skinnytaste.com/roasted-parmesan-green-beans/
in -  https://www.skinnytaste.com/roasted-red-kuri-coconut-curry-soup/
-url:  https://www.skinnytaste.com/roasted-red-kuri-coconut-curry-soup/
in -  https://www.skinnytaste.com/easy-rosemary-garlic-parmesan-biscuits/
-url:  https://www.skinnytaste.com/easy-rosemary-garlic-parmesan-biscuits/
in -  https://www.skinnytaste.com/pumpkin-hazelnut-flaugnarde-clafoutis/
-url:  https://www.skinnytaste.com/pumpkin-hazelnut-flaugnarde-clafoutis/
in -  https://www.skinnytaste.com/skinny-buttermilk-mashed-potatoes-with/
-url:  https://www.skinnyta

-url:  https://www.skinnytaste.com/cinnamon-apple-spiced-oatmeal/
in -  https://www.skinnytaste.com/baked-chipotle-sweet-potato-fries/
-url:  https://www.skinnytaste.com/baked-chipotle-sweet-potato-fries/
in -  https://www.skinnytaste.com/cranberry-pineapple-sauce/
-url:  https://www.skinnytaste.com/cranberry-pineapple-sauce/
in -  https://www.skinnytaste.com/low-fat-pomegranate-scones/
-url:  https://www.skinnytaste.com/low-fat-pomegranate-scones/
in -  https://www.skinnytaste.com/coconut-curry-butternut-squash-soup/
-url:  https://www.skinnytaste.com/coconut-curry-butternut-squash-soup/
in -  https://www.skinnytaste.com/pumpkin-spice-pancakes-with-pumpkin/
-url:  https://www.skinnytaste.com/pumpkin-spice-pancakes-with-pumpkin/
in -  https://www.skinnytaste.com/pumpkin-smoothie/
-url:  https://www.skinnytaste.com/pumpkin-smoothie/
in -  https://www.skinnytaste.com/skinny-pumpkin-spiced-latte/
-url:  https://www.skinnytaste.com/skinny-pumpkin-spiced-latte/
in -  https://www.skinnytaste

-url:  https://www.skinnytaste.com/stuffed-red-bliss-potatoes-3-pts/
in -  https://www.skinnytaste.com/zucchini-and-tomato-salad-0-ww-pts/
-url:  https://www.skinnytaste.com/zucchini-and-tomato-salad-0-ww-pts/
in -  https://www.skinnytaste.com/baby-greens-with-blackberries-pecans/
-url:  https://www.skinnytaste.com/baby-greens-with-blackberries-pecans/
in -  https://www.skinnytaste.com/ratatouille-0-pts/
-url:  https://www.skinnytaste.com/ratatouille-0-pts/
in -  https://www.skinnytaste.com/huevos-pericos-3-ww-points/
-url:  https://www.skinnytaste.com/huevos-pericos-3-ww-points/


In [29]:
# 레시피 내용 가져오기

url = 'https://www.skinnytaste.com/easiest-slow-cooker-sweet-potatoes/'
contents = dict()
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

# 제목
title = soup.find('div', {'class': 'post-title'})
if title:
    title = title.contents[1].text
else:
    pass

title_index = int(title.find('Recipe'))

if title_index > 0:
    title = title[:title_index]
else:
    pass

# 레시피 내용
recipes_list = list()
recipe_para = soup.find_all('ul', {'class': 'wprm-recipe-instructions'})
for recipes in recipe_para:
    if recipes:
        recipe = recipes.find_all('li', {'class': 'wprm-recipe-instruction'})
        for r in recipe:
            step = str(int(r.get('id').split('-')[-1]) + 1)
            recipes_list.append(step + '. ' + r.text)
    else:
        pass
    
# 재료
ingredients_list = list()
ingredients = soup.find_all('li', {'class': 'wprm-recipe-ingredient'})
for ingredient in ingredients:
    if ingredient:
        ingredients_list.append(ingredient.text)
    else:
        pass

# 조리시간
cooktime_list = list()
cooktimes = soup.find_all('span', {'class': 'wprm-recipe-time wprm-block-text-normal'})
if cooktimes:
    cooktimes = cooktimes[-1]
    for cooktime in cooktimes:
        if cooktime:
            cooktime_list.append(cooktime.text)
        else:
            pass
cooktime_list = ''.join(cooktime_list)

# 분량
servings = soup.find('span', {'class': 'wprm-recipe-servings-with-unit'})
if servings:
    servings = servings.text
else:
    pass

# 영양정보
nut_list = list()
nutrients = soup.find_all('span', {'class': 'wprm-meta-value'})
for nutrient in nutrients:
    if nutrient:
        nut_list.append(nutrient.text)
    else:
        pass

calories = ''
carbs = ''
protein = ''
total_fat = ''


if nut_list:
    calories = nut_list[0].split(' ')[0] + 'kcal'
    carbs = nut_list[2].split(' ')[0] + 'g'
    protein = nut_list[1].split(' ')[0] + 'g'
    total_fat = nut_list[3].split(' ')[0] + 'g'
else:
    pass


# 이미지
image = soup.find('div', {'class', 'wprm-recipe-image wprm-block-image-normal'})
if image:
    image = image.contents[0].get('data-lazy-srcset').split(' ')[2]
else:
    pass

if recipes_list != {}:
    # if comments_num > 4:
    # 주소 (사이트)
    contents['site'] = url
    # 제목
    contents['title'] = title
    # 재료
    contents['ingredients'] = ingredients_list
    # 조리시간
    contents['time'] = cooktime_list
    # 분량
    contents['serving'] = servings
    # 레시피 내용
    contents['recipe'] = recipes_list
    # 칼로리
    contents['calories'] = calories
    # 탄수화물
    contents['carbs'] = carbs
    # 단백질
    contents['protein'] = protein
    # 지방
    contents['total_fat'] = total_fat
    # # 댓글
    # contents['comments'] = com_list
    # 이미지
    contents['image'] = image
    # else:
    #     pass
else:
    contents = ''
print(contents)
# print('-url: ', url)

{'site': 'https://www.skinnytaste.com/easiest-slow-cooker-sweet-potatoes/', 'title': 'Easiest Slow Cooker Sweet Potatoes', 'ingredients': ['4 medium sweet potatoes, about 7 oz each, skin on'], 'time': '8 hrs', 'serving': '4 servings', 'recipe': ['1. Wash and dry the sweet potatoes.', "2. Place in the slow cooker, cover and cook on low for 8 hours, until they are tender. I don't recommend doing this one on high without water. ", '3. When the potato is done, season with salt and pepper, and top with butter, cinnamon, chili, or whatever you like on your sweet potatoes.'], 'calories': '208kcal', 'carbs': '48g', 'protein': '3g', 'total_fat': '0.5g', 'image': 'https://www.skinnytaste.com/wp-content/uploads/2018/11/Slow-Cooker-Sweet-Potatoes-1-500x750.jpg'}
